In [3]:
# LIBRARY MANAGEMENT SYSTEM

In [4]:
import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3
from datetime import datetime, timedelta

def initialize_database():
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS library (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        author TEXT,
        category TEXT,
        is_checked_out INTEGER DEFAULT 0,
        due_date TEXT
    )
    """)
    conn.commit()

    cursor.execute("SELECT COUNT(*) FROM library")
    if cursor.fetchone()[0] == 0:
        static_data = [
            ("The Great Gatsby", "F. Scott Fitzgerald", "Book"),
            ("To Kill a Mockingbird", "Harper Lee", "Book"),
            ("1984", "George Orwell", "Book"),
            ("Moby Dick", "Herman Melville", "Book"),
            ("Pride and Prejudice", "Jane Austen", "Book"),
            ("Time Magazine - Nov 2024", "Various", "Magazine"),
            ("National Geographic - Oct 2024", "Various", "Magazine"),
            ("Forbes - Dec 2024", "Various", "Magazine"),
            ("The Economist - Jan 2024", "Various", "Magazine"),
            ("Science Journal - Vol 23", "Various", "Magazine"),
            ("Inception", "Christopher Nolan", "DVD"),
            ("The Dark Knight", "Christopher Nolan", "DVD"),
            ("Interstellar", "Christopher Nolan", "DVD"),
            ("The Matrix", "Wachowskis", "DVD"),
            ("Avatar", "James Cameron", "DVD"),
            ("Sherlock Holmes", "Arthur Conan Doyle", "Book"),
            ("The Hobbit", "J.R.R. Tolkien", "Book"),
            ("War and Peace", "Leo Tolstoy", "Book"),
            ("Ulysses", "James Joyce", "Book"),
            ("The Catcher in the Rye", "J.D. Salinger", "Book"),
        ]
        cursor.executemany("INSERT INTO library (title, author, category) VALUES (?, ?, ?)", static_data)
        conn.commit()
    return conn

class LibraryApp:
    def __init__(self, root, conn):
        self.root = root
        self.conn = conn
        self.cursor = conn.cursor()
        self.root.title("Library Management System")
        self.create_widgets()
        self.load_items()

    def create_widgets(self):
        tk.Label(self.root, text="Title").grid(row=0, column=0, padx=5, pady=5)
        self.title_entry = tk.Entry(self.root, width=30)
        self.title_entry.grid(row=0, column=1)

        tk.Label(self.root, text="Author").grid(row=1, column=0, padx=5, pady=5)
        self.author_entry = tk.Entry(self.root, width=30)
        self.author_entry.grid(row=1, column=1)

        tk.Label(self.root, text="Category").grid(row=2, column=0, padx=5, pady=5)
        self.category_entry = tk.Entry(self.root, width=30)
        self.category_entry.grid(row=2, column=1)

        tk.Button(self.root, text="Add Item", command=self.add_item).grid(row=3, column=0, padx=5, pady=5)
        tk.Button(self.root, text="Checkout Item", command=self.checkout_item).grid(row=3, column=1, padx=5, pady=5)
        tk.Button(self.root, text="Return Item", command=self.return_item).grid(row=4, column=0, padx=5, pady=5)
        tk.Button(self.root, text="Search Items", command=self.search_items).grid(row=4, column=1, padx=5, pady=5)
        tk.Button(self.root, text="Delete Item", command=self.delete_item).grid(row=5, column=0, columnspan=2, pady=10)

        self.tree = ttk.Treeview(self.root, columns=("ID", "Title", "Author", "Category", "Status", "Due Date"), show="headings")
        self.tree.heading("ID", text="ID")
        self.tree.heading("Title", text="Title")
        self.tree.heading("Author", text="Author")
        self.tree.heading("Category", text="Category")
        self.tree.heading("Status", text="Status")
        self.tree.heading("Due Date", text="Due Date")
        self.tree.column("ID", width=50)
        self.tree.grid(row=6, column=0, columnspan=2, padx=10, pady=10)

    def add_item(self):
        title = self.title_entry.get()
        author = self.author_entry.get()
        category = self.category_entry.get()
        if not title or not category:
            messagebox.showerror("Error", "Title and Category are required!")
            return
        self.cursor.execute("INSERT INTO library (title, author, category) VALUES (?, ?, ?)", (title, author, category))
        self.conn.commit()
        self.load_items()
        messagebox.showinfo("Success", "Item added successfully!")

    def checkout_item(self):
        item_id = self.get_selected_item()
        if not item_id:
            return
        self.cursor.execute("SELECT is_checked_out FROM library WHERE id = ?", (item_id,))
        if self.cursor.fetchone()[0]:
            messagebox.showerror("Error", "Item is already checked out!")
        else:
            due_date = (datetime.now() + timedelta(days=14)).strftime("%Y-%m-%d")
            self.cursor.execute("UPDATE library SET is_checked_out = 1, due_date = ? WHERE id = ?", (due_date, item_id))
            self.conn.commit()
            self.load_items()
            messagebox.showinfo("Success", "Item checked out successfully!")

    def return_item(self):
        item_id = self.get_selected_item()
        if not item_id:
            return
        self.cursor.execute("SELECT is_checked_out, due_date FROM library WHERE id = ?", (item_id,))
        is_checked_out, due_date = self.cursor.fetchone()
        if not is_checked_out:
            messagebox.showerror("Error", "Item is not checked out!")
        else:
            overdue_days = max(0, (datetime.now() - datetime.strptime(due_date, "%Y-%m-%d")).days)
            fine = overdue_days
            self.cursor.execute("UPDATE library SET is_checked_out = 0, due_date = NULL WHERE id = ?", (item_id,))
            self.conn.commit()
            self.load_items()
            messagebox.showinfo("Success", f"Item returned! Fine: ${fine}" if fine else "Item returned successfully!")

    def delete_item(self):
        item_id = self.get_selected_item()
        if not item_id:
            return
        self.cursor.execute("DELETE FROM library WHERE id = ?", (item_id,))
        self.conn.commit()
        self.load_items()
        messagebox.showinfo("Success", "Item deleted successfully!")

    def search_items(self):
        query = self.title_entry.get()
        if not query:
            messagebox.showerror("Error", "Enter a search term!")
            return
        self.cursor.execute("SELECT * FROM library WHERE title LIKE ? OR author LIKE ? OR category LIKE ?", 
                            (f"%{query}%", f"%{query}%", f"%{query}%"))
        self.display_items(self.cursor.fetchall())

    def load_items(self):
        self.cursor.execute("SELECT * FROM library")
        self.display_items(self.cursor.fetchall())

    def display_items(self, rows):
        self.tree.delete(*self.tree.get_children())
        for row in rows:
            status = "Checked Out" if row[4] else "Available"
            self.tree.insert("", tk.END, values=(row[0], row[1], row[2], row[3], status, row[5] if row[5] else "N/A"))

    def get_selected_item(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showerror("Error", "No item selected!")
            return None
        return self.tree.item(selected[0], "values")[0]

if __name__ == "__main__":
    conn = initialize_database()
    root = tk.Tk()
    app = LibraryApp(root, conn)
    root.mainloop()